LULC Base Create code  
Creates the base LST grid for the LULC information to be added to  
Change the CITY_COUNTRY variable and then run

In [1]:
#Manual entries
CITY_COUNTRY = "BIJAPUR_INDIA"

#aqua and terra grid same, doesn't matter which one you use
#SATELLITE = 'MOD11A2' #TERRA
SATELLITE = 'MYD11A2' #AQUA

In [2]:
#import required packages
import os
import pandas as pd
import earthpy as et
import pickle
#for the reprojecting
import pyproj
from pyproj import CRS, Proj
from pyproj import Transformer
import cartopy
import cartopy.crs as ccrs
from shapely.geometry import Point, LineString, Polygon
from pyhdf.SD import SD, SDC
import datetime as dt
import scipy
from scipy import interpolate
import geopandas as gpd
import numpy as np
import re
import fnmatch  #for finding other file when city+rural isn't all in the main one (bulawayo)


#load in csv of city lons and lats
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT'))
CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon.xlsx', index_col=None)
#CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon2.xlsx', index_col=None)

#set the city latitude and longitudes and city name
City_Lat = CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['CITY_COUNTRY'] == CITY_COUNTRY]['Lat'].values[0]
City_Lon = CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['CITY_COUNTRY'] == CITY_COUNTRY]['Lon'].values[0]
City_name = CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['CITY_COUNTRY'] == CITY_COUNTRY]['City'].values[0]

#Area to look at will be the city centre +/- 0.3/0.4 degs depending on city size
min_lat = City_Lat - 0.3
max_lat = City_Lat + 0.3
min_lon = City_Lon - 0.3
max_lon = City_Lon + 0.3

#define function to create coordinate grid
def grid_create(file_name):
    # Identify the data field- use the LST day but grid is same for all data
    DATAFIELD_NAME = 'LST_Day_1km'

    GRID_NAME = 'MODIS_Grid_8Day_1km_LST'

    hdf = SD(file_name, SDC.READ)

    # Read dataset.
    data2D = hdf.select(DATAFIELD_NAME)
    data = data2D[:,:].astype(np.float64)

    # Read global attribute.
    fattrs = hdf.attributes(full=1)
    ga = fattrs["StructMetadata.0"]
    gridmeta = ga[0]

    # Construct the grid.  Required information in global attribute called 'StructMetadata.0'
    
    ul_regex = re.compile(r'''UpperLeftPointMtrs=\(
                                      (?P<upper_left_x>[+-]?\d+\.\d+)
                                  ,
                                      (?P<upper_left_y>[+-]?\d+\.\d+)
                                      \)''', re.VERBOSE)
    match = ul_regex.search(gridmeta)
    x0 = np.float(match.group('upper_left_x')) 
    y0 = np.float(match.group('upper_left_y')) 

    lr_regex = re.compile(r'''LowerRightMtrs=\(
                                      (?P<lower_right_x>[+-]?\d+\.\d+)
                                      ,
                                      (?P<lower_right_y>[+-]?\d+\.\d+)
                                      \)''', re.VERBOSE)
    match = lr_regex.search(gridmeta)
    x1 = np.float(match.group('lower_right_x')) 
    y1 = np.float(match.group('lower_right_y')) 
    ny, nx = data.shape
    xinc = (x1 - x0) / nx
    yinc = (y1 - y0) / ny

    x = np.linspace(x0, x0 + xinc*nx, nx)
    y = np.linspace(y0, y0 + yinc*ny, ny)
    xv, yv = np.meshgrid(x, y)

    #transform to coordinates
    transformer = Transformer.from_crs("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext", "EPSG:4326")
    lat, lon = transformer.transform(xv, yv)

    #create dataframe of the coordinates
    Lon_list = lon.flatten()
    Lat_list = lat.flatten()

    df = pd.DataFrame(list(zip(Lat_list, Lon_list)), 
                   columns =['Latitude', 'Longitude']) 

    #Create dataframe of the required area
    df_subset = df[(df.Latitude > min_lat) & (df.Latitude < max_lat) & (df.Longitude > min_lon) & (df.Longitude < max_lon)]

    return df_subset

In [3]:
#Read daylist (could have used night, doesn't matter), pickle files back in from image select code. 
#Then select random image

##Set path to chosen satellite
os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, SATELLITE))
#with open('Day_list.pkl', 'rb') as f:
#    Day_list = pickle.load(f)

    #select file, doesn't matter which they are all on same grid
#file_name = Day_list[10]
file_name = 'MYD11A2.A2021185.h25v07.006.2021194055129.hdf'

LULC_base = grid_create(file_name)

if CITY_COUNTRY == "BULAWAYO_ZIMBABWE":
#add in other files for cities where the rural extent goes outside of the grid box
    #Set path to chosen satellite
    os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, SATELLITE,'CITY_TOP'))
    
    #extract the julian date of the main filename
    yeardoy = file_name.split('.')[1][1:] 

    #find the filename which contains this in the top of city files
    for file in os.listdir('.'):
        if fnmatch.fnmatch(file, '*{}*'.format(yeardoy)):
            top_file_name = file    
    
    top_file_df = grid_create(top_file_name)

    LULC_base = LULC_base.append(top_file_df).reset_index(drop = True)
    
#save to csv to be used as the base to grid the LULC data to
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','LULC_bases'))
LULC_base.to_csv(r'LULC_base_{}.csv'.format(CITY_COUNTRY), index = False)


In [10]:
#Work out the local epsg and add these to the list of cities
#only need to do this once
#save as csv

def utm_zoner(lon, lat):
    utm_lon = lon+180
    utm_zone = int(np.ceil(utm_lon/6))
    south_hem =''
    if lat<0:
        south_hem = ' +south'
    proj_str = f'+proj=utm +zone={utm_zone}{south_hem}'
    return proj_str

local_epsg_list = []
for i in range(len(CITY_COUNTRY_lat_lon)):
    City_Lon = CITY_COUNTRY_lat_lon['Lon'].values[i]
    City_Lat = CITY_COUNTRY_lat_lon['Lat'].values[i]
    local_utm = CRS.from_proj4(utm_zoner(City_Lon, City_Lat))
    local_epsg = local_utm.to_epsg()
    local_epsg_list.append(local_epsg)

CITY_COUNTRY_lat_lon['Local_epsg'] = local_epsg_list 

#set path to save into and save as csv
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT'))
CITY_COUNTRY_lat_lon.to_csv(r'CITY_COUNTRY_lat_lon2.csv', index = False)

#also save as excel so have epsg for later
CITY_COUNTRY_lat_lon.to_excel("CITY_COUNTRY_lat_lon2.xlsx") 
